# 🏦 Como Empresas Financeiras avaliam seu crédito?
### Análise Preditiva de Score de Crédito desenvolvido em Python e Jupyter Notebook
### ML Pipeline Completo: Da Análise Exploratória à API em Produção

OBJETIVO
- Criar banco de dados SQLite com dados fictícios realistas
- Gerar 100.000 registros de clientes com 16 colunas
- Aplicar lógica de negócio para criar target (inadimplente)
- Salvar backup em CSV
- Validar integridade dos dados

SCHEMA DO BANCO
- 4 categorias de dados (Demográfico, Financeiro, Crédito, Target)
- Total: 16 colunas
- Target: inadimplente (0 = Adimplente, 1 = Inadimplente)


In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

print("🗄️ CRIAÇÃO DO BANCO DE DADOS SQLite")
print(f"📅 Data de execução: {datetime.now().strftime('%d/%m/%Y %H:%M')}")

# Config
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

N_CLIENTES = 100000 

DB_PATH = 'data/database.db'
CSV_BACKUP_PATH = 'data/clientes_backup.csv'

print(f"\n⚙️ CONFIGURAÇÕES:")
print(f"   • Número de clientes: {N_CLIENTES:}")
print(f"   • Seed aleatória: {RANDOM_SEED}")
print(f"   • Banco SQLite: {DB_PATH}")
print(f"   • Backup CSV: {CSV_BACKUP_PATH}")

# Gerar dados fictícios
print(f"\n🔄 GERANDO {N_CLIENTES} registros de clientes...")

In [ ]:
# 1. Identificação e perfil demográfico
print("\n1️⃣ Categoria: Identificação e Perfil Demográfico")

dados = {
  'id_cliente': list(range(1, N_CLIENTES + 1)),

  'idade': np.clip(
    np.random.normal(40, 15, N_CLIENTES).astype(int),
    18,80
  ),

  'genero': np.random.choice(
    ['M', 'F'],
    N_CLIENTES,
    p=[0.48, 0.52]
  ),

  'estado_civil': np.random.choice(
    ['Solteiro', 'Casado', 'Divorciado', 'Viúvo'],
    N_CLIENTES,
    p=[0.35, 0.45, 0.15, 0.05]
  ),

  'escolaridade': np.random.choice(
    ['Fundamental', 'Ensino Médio', 'Ensino Superior', 'Pós-graduação'],
    N_CLIENTES,
    p=[0.20, 0.35, 0.35, 0.10]
  ),

  'estado': np.random.choice(
    ['SP', 'RJ', 'MG', 'RS', 'BA', 'PR', 'SC', 'PE', 'CE', 'GO'],
    N_CLIENTES,
    p=[0.25, 0.15, 0.10, 0.08, 0.08, 0.07, 0.06, 0.06, 0.05, 0.10]
  )
}

print(f"✅ Gerados: id_cliente, idade, genero, estado_civil, escolaridade, estado")

# 2. Dados Financeiros
print("\n2️⃣ Categoria: Dados Financeiros")

dados['renda_anual'] = np.random.lognormal(10.4, 0.8, N_CLIENTES)

dados['valor_patrimonio'] = (
  dados['renda_anual'] * np.random.uniform(1.5, 5, N_CLIENTES) + np.random.lognormal(10, 1.5, N_CLIENTES)
)

prob_imovel = np.clip(dados['valor_patrimonio'] / 300000, 0.2, 0.7)
dados['possui_imovel_proprio'] = np.random.binomial(1, prob_imovel)

prob_carro  = np.clip(dados['renda_anual'] / 100000, 0.3, 0.8)
dados['possui_carro'] = np.random.binomial(1, prob_carro)

dados['ocupacao'] = np.random.choice(
  ['CLT', 'Autônomo', 'Empresário', 'Desempregado'],
  N_CLIENTES,
  p=[0.60, 0.25, 0.10, 0.05]
)

print(f"✅ Gerados: renda_anual, valor_patrimonio, possui_imovel_proprio,")
print(f"possui_carro, ocupacao")

# 3. Histórico de Crédito e Comportamento
print("\n3️⃣ Categoria: Histórico de Crédito e Comportamento")

dados['numero_dependentes'] = np.random.choice(
  [0, 1, 2, 3, 4],
  N_CLIENTES,
  p=[0.25, 0.30, 0.25, 0.25, 0.05]
)

dados['tempo_emprego_atual'] = np.where(
  dados['ocupacao'] == 'Desempregado',
  0,
  np.random.randint(1, 360, N_CLIENTES)
)

dados['score_serasa_externo'] = np.clip(
  np.random.normal(600, 150, N_CLIENTES).astype(int),
  300, 900
)

dados['utilizacao_limite_cartao'] = np.clip(
  np.random.normal(40, 25, N_CLIENTES),
  0, 100
)

dados['historico_atraso_90_dias'] = np.random.choice(
  [0, 1, 2, 3, 4, 5],
  N_CLIENTES,
  p=[0.70, 0.15, 0.08, 0.04, 0.02, 0.01]
)

print(f"✅ Gerados: numero_dependentes, tempo_emprego_atual,")
print(f"score_serasa_externo, utilizacao_limite_cartao,")
print(f"historico_atraso_90_dias")

# Criar DataFrame
df = pd.DataFrame(dados)

print(f"\n✅ DataFrame criado com {len(df):,} linhas e {len(df.columns)} colunas")